# simple eval


### AgentSpec (may need new name)

Wrapper class with a run method to represent any type of function we want to evaluate
- BaseAgentSpec
  - run() -> TeamResult
- AgentChatSpec
  - run(agent: Team | Agent, task: str | ....), calls agent.run -> TeamResult
  - should track duration and total tokens?



### TaskRunner
- take a list of tasks (task is str or may have multimodal?)
- take a list of team configs 
- run team configs against tasks (in background offline task)
- each EvalRun should produce an EvalResult
  - result from TeamManager.run
- each run should be backed by DB... we should be able to kick off a run in the UI and see its status. 
- Each run should have comprehensive metrics that we can show in UI ... start with what team result already provides
- should be possible to restart a run (we check db for those not completed and rerun them) 
- class TaskRunner
  - run(agent_spec, task) calls agent_spec.run(task)

eval = EvalRunner()
task = ["height of eiffel", "depth of lake sevan"]
agent_specs = [agent, ]
eval.run() 



EvalJudge 
- BaseEvalJudge
  - judge() calls eval_func
  - eval_func () -> EvalResult (status: bool, reason: str, metadata:Any)
    - EvalResult shoud be [{dim: status, value: 0/1, reason:str}]
- LLMEvalJudge
  - eval_func (criteria, agent_results )
    - EvalResut should be
      - {dimension: status, value: 0/1, reason:str, max_value:1}
      - {dimension: trajectory_efficiency, value: 0/10, reason:str, max_value+10}
    - 

judge = LLMEvalJudge
criteria = [{dimension: str, prompt: str, }]
results = 

I am working on creating an evaluation platform for my agent traces ...
We should have base classes that allow flexible behaviour but also high level classes that tie to AGS specific abstractions like teams etc 

I should be able to have sample code that defines a list of tasks (which may be string or list of string and images etc), define a list of team configurations, run them teams against the tasks, and then judge the traces, (). Ideally running tasks should be background optimized (posislby threaded or asyn tasks?) as well as runing judgements. All of these things should also be logged to db with status for each run etc There should be a way to kick off a run in the UI and see its status. Each run should have comprehensive metrics that we can show in UI ... start with what team result already provides. It should be possible to restart a run (we check db for those not completed and rerun them).

Dont write any code yet, just reason first and let us come up with a reasonable deisgn. If there is any file you need e.g, a definition of some datamodel, you should let me konw .. do not assume shit that will cuase mypy errors


 
## Task Definition 

We should have a way to define a task as a sequence of test and images and files as needed. 

eval.task?

task = EvalTask(["what is the height of the eiffel tower, only respond with the number in meters])
task = EvalTask(["what is in this image"], Image)

## Run Configuration

We should have a EvalRun configuration that tells us what should be executed when a task is run. IT should be cusotmmizable, e.g., a could just be that this is given to a model and the result returend. 

Should take a task 


BaseEvalRunner
   init()
   run()

LLMEvalRunner
   init(model_client) 
   run(task)
   result = TeamResult(...) 
   return result 


TeamEvalRunner
    run(team_config)
    result = TeamResult(...)


## Judge

THis should take a task result and return some type of structured unified score object. ideally a high level unified score and then granular scores on a set of dimensions.

EvalScore
 - score (int or none)
 - dimensions
   - score (int or none)
   - reason (str)
   - max_value (int)
   - dimension (str)
   - metadata (any)




BaseEvalJudge
   run(task, result) 

LLM judge will provide overall screo and  per dimension scores using criteria
   - eval_func (criteria, agent_results)
   - criteria = [{dimension: str, prompt: str, }]
   - results = EvalScore 


EvalJudgeCriteria
   - dimension (str)
   - prompt (str) 
   - max_value (int)
   - metadata (any) 
  
LLMEvalJudge
   run(task, result, criteria)
    



At the end we should have a system that can take a task, run it against a set of teams, and return a score. 









 ## Task Runner  


 ## Judge   





## Context Vars

I'd like to create infra to maintiain context vars for runs.
- each time a run starts in ws.py or in startstream, we should have a context var that should have things like userid/runid/readable/timetamp. can we have a module that sets this
- tools should be able to read this variable ..import and read this variable and use it? e.,g if they will use this to write files to a directory if it exists ... later my app can scan that dir and then use that share info to front end .. e.,g if a tool geneerates a image we can show thqt in front end


- Update TeamManager 
  - create /userid/run/ ....
  - to scan for changes in /run/ ... and udpate metadata 

In [1]:
import asyncio
from datetime import datetime

# Import necessary components
from autogen_core import ComponentModel
from autogen_core.models import UserMessage
from autogen_agentchat.agents import AssistantAgent 
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Import the evaluation components we created
from autogenstudio.datamodel.eval import EvalTask, EvalRunResult, EvalJudgeCriteria, EvalScore, EvalRunStatus
from autogenstudio.eval.runners import ModelEvalRunner, TeamEvalRunner
from autogenstudio.eval.judges import LLMEvalJudge

# This example demonstrates how to run evaluations with different runners and judge the results

async def run_simple_evaluation():
    """Run a simple evaluation of model and team responses."""
    
    # Step 1: Create a model client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        # api_key is loaded from environment variable OPENAI_API_KEY
    )
    
    # Step 2: Create evaluation tasks
    tasks = [
        EvalTask( 
            name="Eiffel Tower Height",
            description="Answer the question about the Eiffel Tower height",
            input="What is the height of the Eiffel Tower?"
        ),
        EvalTask( 
            name="Lake Tanganyika Depth",
            description="Answer the question about Lake Tanganyika's depth",
            input="What is the depth of Lake Tanganyika?"
        )
    ]
    
    # Step 3: Create evaluation runners
    
    # 3.1: Model runner (direct model access)
    model_runner = ModelEvalRunner(
        model_client=model_client,
        name="Direct Model Runner",
        description="Evaluates tasks by sending them directly to the model"
    )
    
    # 3.2: Team runner (using a simple team with one agent)
    # Create an assistant agent for the team
    agent = AssistantAgent(
        name="research_agent",
        model_client=model_client,
        system_message="You are a helpful assistant"
    )
    
    # Create a team with the agent
    team = RoundRobinGroupChat(participants=[agent], max_turns=3)
    
    # Create a team runner with the team
    team_runner = TeamEvalRunner(
        team=team,
        name="Team Runner",
        description="Evaluates tasks using a team of agents"
    )
    
    # Step 4: Create an LLM judge
    # We use the same model client for simplicity
    judge = LLMEvalJudge(
        model_client=model_client,
        name="Evaluation Judge",
        description="Judges the quality of responses"
    )
    
    # Step 5: Define evaluation criteria
    criteria = [
        EvalJudgeCriteria(
            dimension="accuracy",
            prompt="Evaluate the factual accuracy of the response. Are all facts correct?",
            min_value=0,
            max_value=10
        ),
        EvalJudgeCriteria(
            dimension="completeness",
            prompt="Evaluate how thoroughly the response addresses the question. Does it provide all relevant information?",
            min_value=0,
            max_value=10
        )
    ]
    
    # Step 6: Run evaluations and judge the results
    results = {}
    
    print("\n=== Running Evaluations ===\n")
    
    # Run model evaluations
    print("Running model evaluations...")
    model_results = {}
    for task in tasks:
        print(f"  Evaluating task: {task.name}")
        model_result = await model_runner.run(task)
        model_results[task.task_id] = model_result
        
        # Print model response
        if model_result.status:
            print(f"  Model response: {model_result.result}")
        else:
            print(f"  Model error: {model_result.error}")
    
    # Run team evaluations
    print("\nRunning team evaluations...")
    team_results = {}
    for task in tasks:
        print(f"  Evaluating task: {task.name}")
        team_result = await team_runner.run(task)
        team_results[task.task_id] = team_result
        
        # Print team response
        if team_result.status:
            messages = team_result.result.messages or []
            final_message = messages[-1] if messages else None
            if final_message and final_message is not None:
                print(f"  Team response: {final_message.content[:100]}...")
            else:
                print("  No response from team")
        else:
            print(f"  Team error: {team_result.error}")
    
    # Judge the results
    print("\n=== Judging Results ===\n")
    
    # Judge model results
    print("Judging model results...")
    model_scores = {}
    for task in tasks:
        if task.task_id in model_results and model_results[task.task_id].status:
            print(f"  Judging task: {task.name}")
            model_score = await judge.judge(task, model_results[task.task_id], criteria)
            model_scores[task.task_id] = model_score
            
            # Print scores
            print(f"  Overall score: {model_score.overall_score}")
            for dimension_score in model_score.dimension_scores:
                print(f"    {dimension_score.dimension}: {dimension_score.score} - {dimension_score.reason[:50]}...")
    
    # Judge team results
    print("\nJudging team results...")
    team_scores = {}
    for task in tasks:
        if task.task_id in team_results and team_results[task.task_id].status:
            print(f"  Judging task: {task.name}")
            team_score = await judge.judge(task, team_results[task.task_id], criteria)
            team_scores[task.task_id] = team_score
            
            # Print scores
            print(f"  Overall score: {team_score.overall_score}")
            for dimension_score in team_score.dimension_scores:
                print(f"    {dimension_score.dimension}: {dimension_score.score} - {dimension_score.reason[:50]}...")
    
    # Step 7: Test serialization and deserialization
    print("\n=== Testing Serialization and Deserialization ===\n")
    
    # Serialize model runner
    model_runner_config = model_runner.dump_component()
    print(f"Serialized model runner: {model_runner_config.model_dump()}")
    
    # Deserialize model runner
    deserialized_model_runner = ModelEvalRunner.load_component(model_runner_config)
    print(f"Deserialized model runner: {deserialized_model_runner.name}")
    
    # Serialize judge
    judge_config = judge.dump_component()
    print(f"Serialized judge: {judge_config.model_dump()}")
    
    # Deserialize judge
    deserialized_judge = LLMEvalJudge.load_component(judge_config)
    print(f"Deserialized judge: {deserialized_judge.name}")
    
    # Close the model client
    await model_client.close()
    
    return {
        "model_results": model_results,
        "team_results": team_results,
        "model_scores": model_scores,
        "team_scores": team_scores
    }

# For running in a notebook
results = await run_simple_evaluation()

# Print the results
print("\n=== Evaluation Results ===\n")
for task_id, model_result in results["model_results"].items():
    print(f"Task ID: {task_id}")
    print(f"  Model Result: {model_result.result if model_result.status else model_result.error}") 


=== Running Evaluations ===

Running model evaluations...
  Evaluating task: Eiffel Tower Height
  Model response: messages=[TextMessage(source='model', models_usage=None, metadata={}, content="{'finish_reason': 'stop', 'content': 'The Eiffel Tower stands at a height of approximately 1,083 feet (330 meters) including its antennas. The structure itself is about 1,063 feet (324 meters) tall without the antennas.', 'usage': {'prompt_tokens': 17, 'completion_tokens': 42}, 'cached': False, 'logprobs': None, 'thought': None}", type='TextMessage')] stop_reason=None
  Evaluating task: Lake Tanganyika Depth
  Model response: messages=[TextMessage(source='model', models_usage=None, metadata={}, content="{'finish_reason': 'stop', 'content': 'Lake Tanganyika is one of the deepest lakes in the world, with a maximum depth of approximately 1,470 meters (4,823 feet). It is the second deepest lake in Africa, after Lake Malawi.', 'usage': {'prompt_tokens': 17, 'completion_tokens': 44}, 'cached': False,

In [1]:
import asyncio
from datetime import datetime

# Import necessary components
from autogen_core import ComponentModel
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent 
from autogen_agentchat.teams import RoundRobinGroupChat

# Import the evaluation components
from autogenstudio.datamodel.eval import EvalTask, EvalJudgeCriteria, EvalRunStatus
from autogenstudio.eval.runners import ModelEvalRunner, TeamEvalRunner
from autogenstudio.eval.judges import LLMEvalJudge
from autogenstudio.eval.orchestrator import EvalOrchestrator  # New import

async def run_orchestrated_evaluation():
    """Run a simple evaluation using the EvalOrchestrator."""
    
    # Step 1: Create a model client
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o-mini",
        # api_key is loaded from environment variable OPENAI_API_KEY
    )
    
    # Step 2: Create an orchestrator (without DB for this example)
    orchestrator = EvalOrchestrator()
    
    # Step 3: Create and register tasks
    task_ids = []
    tasks = [
        EvalTask( 
            name="Eiffel Tower Height",
            description="Answer the question about the Eiffel Tower height",
            input="What is the height of the Eiffel Tower?"
        ),
        EvalTask( 
            name="Lake Tanganyika Depth",
            description="Answer the question about Lake Tanganyika's depth",
            input="What is the depth of Lake Tanganyika?"
        )
    ]
    
    for task in tasks:
        task_id = await orchestrator.create_task(task)
        task_ids.append(task_id)
    
    # Step 4: Create and register criteria
    criteria_ids = []
    criteria = [
        EvalJudgeCriteria(
            dimension="accuracy",
            prompt="Evaluate the factual accuracy of the response. Are all facts correct?",
            min_value=0,
            max_value=10
        ),
        EvalJudgeCriteria(
            dimension="completeness",
            prompt="Evaluate how thoroughly the response addresses the question. Does it provide all relevant information?",
            min_value=0,
            max_value=10
        )
    ]
    
    for criterion in criteria:
        criterion_id = await orchestrator.create_criteria(criterion)
        criteria_ids.append(criterion_id)
    
    # Step 5: Create runners
    # Model runner
    model_runner = ModelEvalRunner(
        model_client=model_client,
        name="Direct Model Runner",
        description="Evaluates tasks by sending them directly to the model"
    )
    
    # Team runner
    agent = AssistantAgent(
        name="research_agent",
        model_client=model_client,
        system_message="You are a helpful assistant"
    )
    team = RoundRobinGroupChat(participants=[agent], max_turns=3)
    team_runner = TeamEvalRunner(
        team=team,
        name="Team Runner",
        description="Evaluates tasks using a team of agents"
    )
    
    # Step 6: Create a judge
    judge = LLMEvalJudge(
        model_client=model_client,
        name="Evaluation Judge",
        description="Judges the quality of responses"
    )
    
    # Step 7: Create evaluation runs
    model_run_ids = []
    team_run_ids = []
    
    # Create model runs
    for task_id in task_ids:
        run_id = await orchestrator.create_run(
            task=task_id,
            runner=model_runner,
            judge=judge,
            criteria=criteria_ids,
            name=f"Model Run - Task {task_id}"
        )
        model_run_ids.append(run_id)
    
    # Create team runs
    for task_id in task_ids:
        run_id = await orchestrator.create_run(
            task=task_id,
            runner=team_runner,
            judge=judge,
            criteria=criteria_ids,
            name=f"Team Run - Task {task_id}"
        )
        team_run_ids.append(run_id)
    
    # Step 8: Execute the runs
    print("\n=== Starting Evaluation Runs ===\n")
    
    # Start model runs
    print("Starting model runs...")
    for run_id in model_run_ids:
        await orchestrator.start_run(run_id)
    
    # Start team runs
    print("Starting team runs...")
    for run_id in team_run_ids:
        await orchestrator.start_run(run_id)
    
    # Step 9: Wait for runs to complete
    print("\n=== Waiting for Runs to Complete ===\n")
    
    all_runs = model_run_ids + team_run_ids
    completed = {run_id: False for run_id in all_runs}
    
    while not all(completed.values()):
        for run_id in all_runs:
            if not completed[run_id]:
                status = await orchestrator.get_run_status(run_id)
                if status in [EvalRunStatus.COMPLETED, EvalRunStatus.FAILED, EvalRunStatus.CANCELED]:
                    completed[run_id] = True
                    print(f"Run {run_id} completed with status: {status}")
        
        await asyncio.sleep(1)
    
    # Step 10: Get results
    print("\n=== Evaluation Results ===\n")
    
    # Model results
    print("Model run results:")
    for run_id in model_run_ids:
        run_result = await orchestrator.get_run_result(run_id)
        score_result = await orchestrator.get_run_score(run_id)
        
        print(f"Run ID: {run_id}")
        if run_result and run_result.status:
            print(f"  Response: {run_result.result}")
            
            if score_result:
                print(f"  Overall score: {score_result.overall_score}")
                for dimension_score in score_result.dimension_scores:
                    print(f"    {dimension_score.dimension}: {dimension_score.score}")
                    print(f"    Reason: {dimension_score.reason[:100]}...")
        else:
            print(f"  Error: {run_result.error if run_result else 'No result'}")
    
    # Team results
    print("\nTeam run results:")
    for run_id in team_run_ids:
        run_result = await orchestrator.get_run_result(run_id)
        score_result = await orchestrator.get_run_score(run_id)
        
        print(f"Run ID: {run_id}")
        if run_result and run_result.status:
            messages = run_result.result.messages or []
            final_message = messages[-1] if messages else None
            if final_message and final_message is not None:
                print(f"  Response: {final_message.content[:100]}...")
            
            if score_result:
                print(f"  Overall score: {score_result.overall_score}")
                for dimension_score in score_result.dimension_scores:
                    print(f"    {dimension_score.dimension}: {dimension_score.score}")
                    print(f"    Reason: {dimension_score.reason[:100]}...")
        else:
            print(f"  Error: {run_result.error if run_result else 'No result'}")
    
    # Close the model client
    await model_client.close()
    
    return {
        "task_ids": task_ids,
        "criteria_ids": criteria_ids,
        "model_run_ids": model_run_ids,
        "team_run_ids": team_run_ids
    }

# For running in a notebook
results = await run_orchestrated_evaluation()
print("\n=== Orchestrated Evaluation Results ===\n")
for task_id, run_id in zip(results["task_ids"], results["model_run_ids"]):
    print(f"Task ID: {task_id}, Model Run ID: {run_id}")

2025-04-05 14:44:20.998 | INFO     | autogenstudio.eval.orchestrator:_execute_run:354 - Starting runner for run a48f657e-5057-4f83-a711-3a5f88c5ced5
2025-04-05 14:44:21.011 | INFO     | autogenstudio.eval.orchestrator:_execute_run:354 - Starting runner for run 2c977e89-2e15-4176-9f89-5666b3cf5959
2025-04-05 14:44:21.029 | INFO     | autogenstudio.eval.orchestrator:_execute_run:354 - Starting runner for run 09584e22-4a6d-4663-8ff6-0da3d70226d1
2025-04-05 14:44:21.044 | INFO     | autogenstudio.eval.orchestrator:_execute_run:354 - Starting runner for run 7b4bee41-3fb4-4e3c-babc-9f56e4b42b0f



=== Starting Evaluation Runs ===

Starting model runs...
Starting team runs...

=== Waiting for Runs to Complete ===



2025-04-05 14:44:21.884 | INFO     | autogenstudio.eval.orchestrator:_execute_run:367 - Starting judge for run a48f657e-5057-4f83-a711-3a5f88c5ced5
2025-04-05 14:44:21.899 | INFO     | autogenstudio.eval.orchestrator:_execute_run:367 - Starting judge for run 2c977e89-2e15-4176-9f89-5666b3cf5959
2025-04-05 14:44:24.001 | INFO     | autogenstudio.eval.orchestrator:_execute_run:367 - Starting judge for run 7b4bee41-3fb4-4e3c-babc-9f56e4b42b0f
2025-04-05 14:44:24.232 | INFO     | autogenstudio.eval.orchestrator:_execute_run:377 - Run a48f657e-5057-4f83-a711-3a5f88c5ced5 completed successfully
2025-04-05 14:44:24.750 | INFO     | autogenstudio.eval.orchestrator:_execute_run:367 - Starting judge for run 09584e22-4a6d-4663-8ff6-0da3d70226d1
2025-04-05 14:44:24.848 | INFO     | autogenstudio.eval.orchestrator:_execute_run:377 - Run 2c977e89-2e15-4176-9f89-5666b3cf5959 completed successfully


Run a48f657e-5057-4f83-a711-3a5f88c5ced5 completed with status: EvalRunStatus.COMPLETED
Run 2c977e89-2e15-4176-9f89-5666b3cf5959 completed with status: EvalRunStatus.COMPLETED


2025-04-05 14:44:27.219 | INFO     | autogenstudio.eval.orchestrator:_execute_run:377 - Run 7b4bee41-3fb4-4e3c-babc-9f56e4b42b0f completed successfully


Run 7b4bee41-3fb4-4e3c-babc-9f56e4b42b0f completed with status: EvalRunStatus.COMPLETED


2025-04-05 14:44:28.932 | INFO     | autogenstudio.eval.orchestrator:_execute_run:377 - Run 09584e22-4a6d-4663-8ff6-0da3d70226d1 completed successfully


Run 09584e22-4a6d-4663-8ff6-0da3d70226d1 completed with status: EvalRunStatus.COMPLETED

=== Evaluation Results ===

Model run results:
Run ID: a48f657e-5057-4f83-a711-3a5f88c5ced5
  Response: TaskResult(messages=[TextMessage(content="{'finish_reason': 'stop', 'content': 'The Eiffel Tower stands at approximately 1,083 feet (330 meters) tall, including its antennas. Without the antennas, its height is about 1,063 feet (324 meters).', 'usage': {'prompt_tokens': 17, 'completion_tokens': 39}, 'cached': False, 'logprobs': None, 'thought': None}", source='model', models_usage=None, metadata={}, type='TextMessage')], stop_reason=None)
  Overall score: 9.25
    accuracy: 10.0
    Reason: The response correctly states that the Eiffel Tower stands at approximately 1,083 feet (330 meters) ...
    completeness: 8.5
    Reason: The response provides a clear and accurate height of the Eiffel Tower with two measurements: approxi...
Run ID: 2c977e89-2e15-4176-9f89-5666b3cf5959
  Response: TaskResult(m

In [2]:
results

{'task_ids': ['0f12705d-af15-40f3-ba61-aab4035095d9',
  '64cdbf50-c92a-4d18-94b0-d2c745b6c2c0'],
 'criteria_ids': ['21c11ae1-f481-4bc3-a358-d4597cda6a1c',
  'b9ed83c3-9bd1-4bf4-8ae0-acffb8310c32'],
 'model_run_ids': ['a48f657e-5057-4f83-a711-3a5f88c5ced5',
  '2c977e89-2e15-4176-9f89-5666b3cf5959'],
 'team_run_ids': ['09584e22-4a6d-4663-8ff6-0da3d70226d1',
  '7b4bee41-3fb4-4e3c-babc-9f56e4b42b0f']}

In [ ]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient, OpenAIChatCompletionClient
from autogen_ext.models.anthropic import AnthropicChatCompletionClient
from autogen_core.models import ModelInfo

model_client=OpenAIChatCompletionClient(
            model="gpt-4o-mini",
        )
print(model_client.dump_component().model_dump_json())


az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="{your-azure-deployment}",
    model="gpt-4o",
    api_version="2024-06-01",
    azure_endpoint="https://{your-custom-endpoint}.openai.azure.com/",
    api_key="sk-...",
)
print(az_model_client.dump_component().model_dump_json())

anthropic_client = AnthropicChatCompletionClient(
        model="claude-3-sonnet-20240229",
        api_key="your-api-key",  # Optional if ANTHROPIC_API_KEY is set in environment
    )
print(anthropic_client.dump_component().model_dump_json())

mistral_vllm_model = OpenAIChatCompletionClient(
        model="TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
        base_url="http://localhost:1234/v1",
        model_info=ModelInfo(vision=False, function_calling=True, json_output=False, family="unknown", structured_output=True),
    )
print(mistral_vllm_model.dump_component().model_dump_json())


In [5]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo

model_client = OpenAIChatCompletionClient(
    model="gemini-2.0-flash-lite",
    model_info=ModelInfo(vision=True, function_calling=True, json_output=True, family="unknown", structured_output=True)
    # api_key="GEMINI_API_KEY",
)

response = await model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(response)
await model_client.close()


finish_reason='stop' content='The capital of France is Paris.\n' usage=RequestUsage(prompt_tokens=7, completion_tokens=8) cached=False logprobs=None thought=None


In [11]:
from typing import Literal

from pydantic import BaseModel
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
 

# The response format for the agent as a Pydantic base model.
class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]


# Create an agent that uses the OpenAI GPT-4o model with the custom response format.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    response_format=AgentResponse,  # type: ignore
)

# Send a message list to the model and await the response.
messages = [
    UserMessage(content="I am happy today thinking of flowers.  Format as structured json", source="user"),
]
response = await model_client.create(messages=messages)
assert isinstance(response.content, str)
print(response)

# parsed_response = AgentResponse.model_validate_json(response.content)
# print(parsed_response.thoughts)
# print(parsed_response.response)

# # Close the connection to the model client.
# await model_client.close()


finish_reason='stop' content='{"thoughts":"The user is expressing a positive emotion related to flowers, which can symbolize beauty and happiness. It\'s a good idea to explore this topic further, perhaps by asking about their favorite flowers or memories associated with them.","response":"happy"}' usage=RequestUsage(prompt_tokens=82, completion_tokens=50) cached=False logprobs=None thought=None


In [ ]:
import os
import requests
import asyncio
from typing import Dict, Any, Optional, Union, List
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient


async def get_svcdeep_diagnosis(svcdata: str) -> Dict[str, Any]:
    """
    Get service deep diagnosis for an incident.
    
    Args:
        svcdata: Incident data and scenario name in JSON format
    
    Returns:
        Dictionary with diagnosis results or error information
    """
    base_url: Optional[str] = os.getenv("SVCDEEP_DIAGNOSIS_URL")
    
    try:
        response = requests.post(
            base_url, 
            json=svcdata)

        response.raise_for_status()
        data: Dict[str, Any] = response.json()

        return {
            "success": True,
            "location": data["location"]["name"],
            "temperature_c": data["current"]["temp_c"],
            "condition": data["current"]["condition"]["text"]
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }


async def get_icm_info(incident: str) -> Dict[str, Any]:
    """
    Get ICM information for an incident.
    
    Args:
        incident: Incident identifier or data
    
    Returns:
        Dictionary with ICM information or error details
    """
    try:
        # Implementation of ICM info retrieval logic would go here
        return {
            "success": True,
            "incident_id": incident,
            "status": "Active",
            "severity": "2"
        }
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }


 


async def main() -> None:
    """
    Main function to run the diagnosis agent.
    """
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o",
    )
    
    diagnosis_agent = AssistantAgent(
        name="diagnosis_advisor",
        model_client=model_client,
        tools=[get_svcdeep_diagnosis, get_icm_info],
        system_message="You are a diagnosis advisor who can provide possible causation insights and information.", 
        model_client_stream=True,
    )
    icm_analysis = AssistantAgent(
        name="diagnosis_advisor",
        model_client=model_client,
        tools=[get_svcdeep_diagnosis, get_icm_info],
        system_message="You are a diagnosis advisor who can provide possible causation insights and information.",
        reflect_on_tool_use=True,
        model_client_stream=True,
    )
    
    try:
        await Console(agent.run_stream(task="Analyze incident ICM12345 and provide diagnosis."))
    finally:
        # Ensure connection is closed even if an error occurs
        await model_client.close()

# if runninng in a python script
# if __name__ == "__main__":
#     asyncio.run(main())

await main()

---------- user ----------
Analyze incident ICM12345 and provide diagnosis.


---------- diagnosis_advisor ----------
[FunctionCall(id='call_f97OxytTQCrFVJGu9pomJRXR', arguments='{"incident": "ICM12345"}', name='get_icm_info'), FunctionCall(id='call_T46ZgNs5pwtVEu9p8uxZpclk', arguments='{"svcdata": "{\\"incidentId\\":\\"ICM12345\\"}"}', name='get_svcdeep_diagnosis')]
---------- diagnosis_advisor ----------
[FunctionExecutionResult(content="{'success': True, 'incident_id': 'ICM12345', 'status': 'Active', 'severity': '2'}", name='get_icm_info', call_id='call_f97OxytTQCrFVJGu9pomJRXR', is_error=False), FunctionExecutionResult(content='{\'success\': False, \'error\': "Invalid URL \'None\': No scheme supplied. Perhaps you meant https://None?"}', name='get_svcdeep_diagnosis', call_id='call_T46ZgNs5pwtVEu9p8uxZpclk', is_error=False)]
---------- diagnosis_advisor ----------
For incident ICM12345, here is the basic information retrieved:

- **Incident ID**: ICM12345
- **Status**: Active
- **Severity**: 2

Unfortunately, there was an error when attempting to perform a mor

In [4]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import StdioServerParams, create_mcp_server_session, mcp_server_tools

 
model_client = OpenAIChatCompletionClient(model="gpt-4o", parallel_tool_calls=False)  # type: ignore
params = StdioServerParams(
    command="npx",
    args=["@playwright/mcp@latest"],
    read_timeout_seconds=60,
)
async with create_mcp_server_session(params) as session:
    await session.initialize()
    tools = await mcp_server_tools(server_params=params, session=session)
    print(f"Tools: {[tool.name for tool in tools]}")

    agent = AssistantAgent(
        name="Assistant",
        model_client=model_client,
        tools=tools,  # type: ignore
    )

    termination = TextMentionTermination("TERMINATE")
    team = RoundRobinGroupChat([agent], termination_condition=termination)
    # await Console(
    #     team.run_stream(
    #         task="Go to https://ekzhu.com/, visit the first link in the page, then tell me about the linked page."
    #     )
    # )

    # await model_client.close() 
    

Tools: ['browser_close', 'browser_wait', 'browser_resize', 'browser_file_upload', 'browser_install', 'browser_press_key', 'browser_navigate', 'browser_navigate_back', 'browser_navigate_forward', 'browser_pdf_save', 'browser_snapshot', 'browser_click', 'browser_drag', 'browser_hover', 'browser_type', 'browser_select_option', 'browser_take_screenshot', 'browser_tab_list', 'browser_tab_new', 'browser_tab_select', 'browser_tab_close']


In [6]:
team.dump_component()

ComponentModel(provider='autogen_agentchat.teams.RoundRobinGroupChat', component_type='team', version=1, component_version=1, description='A team that runs a group chat with participants taking turns in a round-robin fashion\n    to publish a message to all.', label='RoundRobinGroupChat', config={'participants': [{'provider': 'autogen_agentchat.agents.AssistantAgent', 'component_type': 'agent', 'version': 1, 'component_version': 1, 'description': 'An agent that provides assistance with tool use.', 'label': 'AssistantAgent', 'config': {'name': 'Assistant', 'model_client': {'provider': 'autogen_ext.models.openai.OpenAIChatCompletionClient', 'component_type': 'model', 'version': 1, 'component_version': 1, 'description': 'Chat completion client for OpenAI hosted models.', 'label': 'OpenAIChatCompletionClient', 'config': {'model': 'gpt-4o'}}, 'tools': [{'provider': 'autogen_ext.tools.mcp.StdioMcpToolAdapter', 'component_type': 'tool', 'version': 1, 'component_version': 1, 'description': 'Al